In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import keras 
import cv2
from PIL import Image
import os
import zipfile
import logging
import json

from cleverhans.future.tf2.attacks import projected_gradient_descent, fast_gradient_method

import alibi_detect.ad as adv
from alibi_detect.datasets import fetch_traffic_signs
from alibi_detect.models.losses import loss_adv_vae
from alibi_detect.ad import AdversarialVAE
from alibi_detect.utils.saving import save_detector, load_detector, save_tf_model, load_tf_model
from alibi_detect.utils.visualize import plot_instance_score

Using TensorFlow backend.
ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [2]:
from inspect import signature
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score, recall_score, accuracy_score, precision_recall_curve, f1_score
import matplotlib
import seaborn as sns
from creme.stats import Mean, Var
%matplotlib inline
from scipy.stats import percentileofscore
from importlib import reload

In [6]:
from sklearn.metrics import confusion_matrix
import foolbox
%set_env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [4]:
train, test = tf.keras.datasets.mnist.load_data()
X_train, y_train = train
X_test, y_test = test

X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000, 10)


In [11]:
    inputs = Input(shape=(X_train.shape[1:]))
    x = Conv2D(filters=64, kernel_size=(2, 2), padding='same', activation='relu')(inputs)
    x = Conv2D(filters=64, kernel_size=(2, 2), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)
    
    x = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(x)
    x = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    logits = Dense(10, name='logits')(x)
    outputs = Activation('softmax', name='softmax')(logits)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    # train model

    model.fit(X_train,
              y_train,
              epochs=30,
              batch_size=256,
              verbose=1,
              validation_data=(X_test, y_test))

NameError: name 'Input' is not defined

In [5]:
#json_file = open('model_tmp/model.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#model = tf.keras.models.model_from_json(loaded_model_json)

In [28]:
#model = tf.keras.models.load_model('/home/gio/adversarial_vae/adversarial-vae/models/classifier/mnist/model/model.h5')

In [7]:
mean = X_train.mean(axis=1).mean()
std = X_train.std()

In [8]:
#preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
#preprocessing = dict(mean=X_train.mean(), std=X_train.std())

In [9]:
fmodel = foolbox.models.KerasModel(model, bounds=(0, 1))

NameError: name 'model' is not defined

In [43]:
labels.shape

(16,)

In [44]:
y_test.shape

(10000, 10)

In [45]:


# instantiate model (supports PyTorch, Keras, TensorFlow (Graph and Eager), JAX, MXNet and many more)
#model = models.resnet18(pretrained=True).eval()
#preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)


# get a batch of images and labels and print the accuracy
#images, labels = foolbox.utils.samples(dataset='mnist', batchsize=16, data_format='channels_last', bounds=(0, 1))
#images = images.reshape(images.shape + (1, ))
#print(images.shape)
#print(np.mean(fmodel.forward(images).argmax(axis=-1) == labels))
# -> 0.9375

# apply the attack
print(X_test.shape)
labels = np.argmax(y_test, axis=1)
attack = foolbox.attacks.FGSM(fmodel)
adversarials = attack(X_test, labels)
# if the i'th image is misclassfied without a perturbation, then adversarials[i] will be the same as images[i]
# if the attack fails to find an adversarial for the i'th image, then adversarials[i] will all be np.nan

# Foolbox guarantees that all returned adversarials are in fact in adversarials
print(np.mean(fmodel.forward(adversarials).argmax(axis=-1) == labels))

(10000, 28, 28, 1)


INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassifie

INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
INFO:root:Not running the attack because the original input is already misclassifie

INFO:root:repeating attack with smaller epsilons
INFO:root:repeating attack with smaller epsilons
INFO:root:190 of 10000 attacks completed
INFO:root:repeating attack with smaller epsilons
INFO:root:repeating attack with smaller epsilons
INFO:root:repeating attack with smaller epsilons
INFO:root:repeating attack with smaller epsilons
INFO:root:repeating attack with smaller epsilons
INFO:root:198 of 10000 attacks completed
INFO:root:repeating attack with smaller epsilons
INFO:root:repeating attack with smaller epsilons
INFO:root:repeating attack with smaller epsilons
INFO:root:repeating attack with smaller epsilons
INFO:root:204 of 10000 attacks completed
INFO:root:207 of 10000 attacks completed
INFO:root:repeating attack with smaller epsilons
INFO:root:repeating attack with smaller epsilons
INFO:root:repeating attack with smaller epsilons
INFO:root:216 of 10000 attacks completed
INFO:root:repeating attack with smaller epsilons
INFO:root:repeating attack with smaller epsilons
INFO:root:r

INFO:root:4639 of 10000 attacks completed
INFO:root:4708 of 10000 attacks completed
INFO:root:4761 of 10000 attacks completed
INFO:root:4815 of 10000 attacks completed
INFO:root:4879 of 10000 attacks completed
INFO:root:4930 of 10000 attacks completed
INFO:root:4990 of 10000 attacks completed
INFO:root:5046 of 10000 attacks completed
INFO:root:5114 of 10000 attacks completed
INFO:root:5156 of 10000 attacks completed
INFO:root:5208 of 10000 attacks completed
INFO:root:5264 of 10000 attacks completed
INFO:root:5324 of 10000 attacks completed
INFO:root:5370 of 10000 attacks completed
INFO:root:5418 of 10000 attacks completed
INFO:root:5470 of 10000 attacks completed
INFO:root:5520 of 10000 attacks completed
INFO:root:5570 of 10000 attacks completed
INFO:root:5611 of 10000 attacks completed
INFO:root:5660 of 10000 attacks completed
INFO:root:5706 of 10000 attacks completed
INFO:root:5762 of 10000 attacks completed
INFO:root:5814 of 10000 attacks completed
INFO:root:5868 of 10000 attacks co

INFO:root:9157 of 10000 attacks completed
INFO:root:9166 of 10000 attacks completed
INFO:root:9170 of 10000 attacks completed
INFO:root:9175 of 10000 attacks completed
INFO:root:9182 of 10000 attacks completed
INFO:root:9192 of 10000 attacks completed
INFO:root:9196 of 10000 attacks completed
INFO:root:9200 of 10000 attacks completed
INFO:root:9205 of 10000 attacks completed
INFO:root:9208 of 10000 attacks completed
INFO:root:9212 of 10000 attacks completed
INFO:root:9216 of 10000 attacks completed
INFO:root:9218 of 10000 attacks completed
INFO:root:9221 of 10000 attacks completed
INFO:root:9225 of 10000 attacks completed
INFO:root:9230 of 10000 attacks completed
INFO:root:9238 of 10000 attacks completed
INFO:root:9241 of 10000 attacks completed
INFO:root:9244 of 10000 attacks completed
INFO:root:9246 of 10000 attacks completed
INFO:root:9252 of 10000 attacks completed
INFO:root:9254 of 10000 attacks completed
INFO:root:9257 of 10000 attacks completed
INFO:root:9261 of 10000 attacks co

INFO:root:9643 of 10000 attacks completed
INFO:root:9644 of 10000 attacks completed
INFO:root:9645 of 10000 attacks completed
INFO:root:9646 of 10000 attacks completed
INFO:root:9647 of 10000 attacks completed
INFO:root:9648 of 10000 attacks completed
INFO:root:9649 of 10000 attacks completed
INFO:root:9651 of 10000 attacks completed
INFO:root:9653 of 10000 attacks completed
INFO:root:9655 of 10000 attacks completed
INFO:root:9657 of 10000 attacks completed
INFO:root:9659 of 10000 attacks completed
INFO:root:9662 of 10000 attacks completed
INFO:root:9663 of 10000 attacks completed
INFO:root:9664 of 10000 attacks completed
INFO:root:9665 of 10000 attacks completed
INFO:root:9666 of 10000 attacks completed
INFO:root:9667 of 10000 attacks completed
INFO:root:9668 of 10000 attacks completed
INFO:root:9669 of 10000 attacks completed
INFO:root:9670 of 10000 attacks completed
INFO:root:9671 of 10000 attacks completed
INFO:root:9672 of 10000 attacks completed
INFO:root:9673 of 10000 attacks co

0.0


In [46]:
adversarials.shape

(10000, 28, 28, 1)